## 埋め込み

前回の例では、`vocab_size`の長さを持つ高次元のバッグオブワードベクトルを操作し、低次元の位置表現ベクトルを明示的にスパースなワンホット表現に変換しました。このワンホット表現はメモリ効率が良くありません。さらに、各単語は互いに独立して扱われるため、ワンホットエンコードされたベクトルは単語間の意味的な類似性を表現しません。

このユニットでは、**News AG**データセットを引き続き探求します。まずはデータをロードし、前のユニットからいくつかの定義を取得しましょう。


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### 埋め込みとは？

**埋め込み（embedding）** のアイデアは、単語をその意味を反映した低次元の密なベクトルで表現することです。後ほど、意味のある単語埋め込みを構築する方法について説明しますが、ここでは埋め込みを単語ベクトルの次元を削減する方法として考えてみましょう。

埋め込み層は単語を入力として受け取り、指定された `embedding_size` の出力ベクトルを生成します。ある意味では、これは `Dense` 層に非常に似ていますが、入力としてワンホットエンコードされたベクトルを取る代わりに、単語番号を直接受け取ることができます。

ネットワークの最初の層として埋め込み層を使用することで、バッグオブワード（bag-of-words）モデルから **埋め込みバッグ（embedding bag）** モデルに切り替えることができます。このモデルでは、まずテキスト内の各単語を対応する埋め込みに変換し、その後、`sum`、`average`、`max` などの集約関数をこれらの埋め込み全体に対して計算します。

![5つのシーケンス単語に対する埋め込み分類器を示す画像。](../../../../../translated_images/ja/embedding-classifier-example.b77f021a7ee67eee.webp)

私たちの分類器ニューラルネットワークは以下の層で構成されています：

* `TextVectorization` 層：文字列を入力として受け取り、トークン番号のテンソルを生成します。適切な語彙サイズ `vocab_size` を指定し、使用頻度の低い単語を無視します。入力の形状は1で、出力の形状は $n$ になります。これは、結果として $n$ 個のトークンを得るためで、それぞれが 0 から `vocab_size` の範囲の番号を含みます。
* `Embedding` 層：$n$ 個の番号を受け取り、それぞれの番号を指定された長さ（この例では100）の密なベクトルに変換します。したがって、形状 $n$ の入力テンソルは $n\times 100$ のテンソルに変換されます。
* 集約層：このテンソルの第1軸に沿って平均を計算します。つまり、異なる単語に対応する $n$ 個の入力テンソル全体の平均を計算します。この層を実装するために、`Lambda` 層を使用し、平均を計算する関数を渡します。出力の形状は100になり、これは入力シーケンス全体の数値表現となります。
* 最後の `Dense` 線形分類器。


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` の出力において、**output shape** 列の最初のテンソル次元 `None` はミニバッチサイズを表し、2番目の次元はトークンシーケンスの長さを表します。ミニバッチ内のすべてのトークンシーケンスは異なる長さを持っています。この問題への対処方法については、次のセクションで説明します。

それでは、ネットワークをトレーニングしてみましょう:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **注意** データのサブセットに基づいてベクトライザーを構築していることを理解してください。これはプロセスを高速化するために行われており、その結果、テキスト内のすべてのトークンが語彙に含まれない場合があります。この場合、それらのトークンは無視されるため、精度がわずかに低下する可能性があります。しかし、実際にはテキストのサブセットでも良好な語彙の推定が得られることが多いです。


### 変数シーケンスサイズの扱い

ミニバッチでの学習がどのように行われるかを理解しましょう。上記の例では、入力テンソルの次元は1であり、128個のミニバッチを使用するため、テンソルの実際のサイズは $128 \times 1$ となります。しかし、各文のトークン数は異なります。`TextVectorization` レイヤーを単一の入力に適用すると、テキストのトークン化方法によって返されるトークン数が異なります。


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


しかし、ベクトライザーを複数のシーケンスに適用する場合、矩形の形状を持つテンソルを生成する必要があるため、未使用の要素をPADトークン（この場合はゼロ）で埋めます。


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

ここで埋め込みを見ることができます:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **注意**: パディングの量を最小限に抑えるために、場合によってはデータセット内のすべてのシーケンスを長さの増加順（より正確にはトークン数の増加順）に並べ替えることが理にかなっています。これにより、各ミニバッチが類似した長さのシーケンスを含むようになります。


## セマンティック埋め込み: Word2Vec

前回の例では、埋め込み層が単語をベクトル表現にマッピングする方法を学習しましたが、これらの表現にはセマンティックな意味がありませんでした。類似した単語や同義語が、あるベクトル距離（例えばユークリッド距離）において互いに近いベクトルに対応するようなベクトル表現を学習できると便利です。

そのためには、大量のテキストコレクションを使用して、[Word2Vec](https://en.wikipedia.org/wiki/Word2vec)のような手法で埋め込みモデルを事前学習する必要があります。Word2Vecは、単語の分散表現を生成するために使用される2つの主要なアーキテクチャに基づいています：

 - **連続バグ・オブ・ワーズ** (CBoW): モデルに周囲の文脈から単語を予測させる手法です。ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ が与えられた場合、モデルの目標は $(W_{-2},W_{-1},W_1,W_2)$ から $W_0$ を予測することです。
 - **連続スキップグラム**: CBoWとは逆の手法です。このモデルでは、現在の単語を予測するために周囲の文脈単語のウィンドウを使用します。

CBoWは高速ですが、スキップグラムは処理が遅いものの、頻度の低い単語をより良く表現することができます。

![単語をベクトルに変換するCBoWとスキップグラムのアルゴリズムを示す画像。](../../../../../translated_images/ja/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Googleニュースのデータセットで事前学習されたWord2Vec埋め込みを試すには、**gensim**ライブラリを使用することができます。以下に、'neural'に最も類似した単語を見つける例を示します。

> **Note:** 初めて単語ベクトルを作成する際、ダウンロードに時間がかかる場合があります！


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


単語からベクトル埋め込みを抽出し、分類モデルのトレーニングに使用することもできます。埋め込みは300成分を持っていますが、ここでは明確にするためにベクトルの最初の20成分のみを示します：


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

セマンティック埋め込みの素晴らしい点は、セマンティクスに基づいてベクトルエンコーディングを操作できることです。例えば、*king* と *woman* のベクトル表現にできるだけ近く、*man* からできるだけ遠い単語を見つけるように求めることができます。


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

上記の例では、内部のGenSymマジックが使用されていますが、基本的なロジックは実際には非常に簡単です。埋め込みに関する興味深い点は、埋め込みベクトルに対して通常のベクトル操作を行うことができ、それが単語の**意味**に関する操作を反映するということです。上記の例はベクトル操作の観点で表現することができます：**KING-MAN+WOMAN**に対応するベクトルを計算し（対応する単語のベクトル表現に対して`+`と`-`の操作を行う）、そのベクトルに最も近い単語を辞書から見つけます：


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: *man* と *woman* ベクトルに小さな係数を追加しました - これを削除して何が起こるか試してみてください。

最も近いベクトルを見つけるために、TensorFlowの仕組みを使って、自分のベクトルと語彙内のすべてのベクトルとの距離のベクトルを計算し、その後 `argmin` を使用して最小の単語のインデックスを見つけます。


Word2Vecは単語の意味を表現する優れた方法のように思えますが、以下のような多くの欠点があります：

* CBoWモデルとskip-gramモデルはどちらも**予測型埋め込み**であり、ローカルな文脈のみを考慮します。Word2Vecはグローバルな文脈を活用しません。
* Word2Vecは単語の**形態**、つまり単語の意味が語幹などの異なる部分に依存する可能性があるという事実を考慮しません。

**FastText**はこの2つ目の制約を克服しようと試み、Word2Vecを基に各単語とその中に含まれる文字n-gramのベクトル表現を学習します。これらの表現値は、各トレーニングステップで1つのベクトルに平均化されます。この方法は事前学習に多くの追加計算を必要としますが、単語埋め込みがサブワード情報をエンコードできるようにします。

もう1つの手法である**GloVe**は、単語埋め込みに対して異なるアプローチを採用しており、単語-文脈行列の因数分解に基づいています。まず、異なる文脈での単語の出現回数をカウントする大きな行列を構築し、その後、この行列を再構築誤差を最小化する形で低次元に表現しようとします。

gensimライブラリはこれらの単語埋め込みをサポートしており、上記のモデル読み込みコードを変更することで、それらを試すことができます。


## Kerasで事前学習済みの埋め込みを使用する

上記の例を修正して、埋め込み層の行列をWord2Vecのようなセマンティック埋め込みで事前に初期化することができます。事前学習済みの埋め込みの語彙とテキストコーパスの語彙は一致しない可能性が高いため、どちらかを選択する必要があります。ここでは、トークナイザーの語彙を使用する場合と、Word2Vecの埋め込みの語彙を使用する場合の2つの選択肢を検討します。

### トークナイザーの語彙を使用する場合

トークナイザーの語彙を使用する場合、語彙の中には対応するWord2Vecの埋め込みが存在する単語もあれば、存在しない単語もあります。語彙サイズが`vocab_size`で、Word2Vecの埋め込みベクトルの長さが`embed_size`の場合、埋め込み層は形状が`vocab_size`$\times$`embed_size`の重み行列で表されます。この行列を語彙を順に処理して埋めていきます:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Word2Vecの語彙に存在しない単語については、ゼロのままにするか、ランダムなベクトルを生成することができます。

次に、事前学習済みの重みを使用して埋め込み層を定義します:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Note**: `Embedding`を作成する際に`trainable=False`を設定していることに注意してください。これは、Embedding層を再学習しないことを意味します。この設定により、精度が若干低下する可能性がありますが、トレーニング速度が向上します。

### 埋め込み語彙の使用

以前のアプローチの問題点は、TextVectorizationとEmbeddingで使用される語彙が異なることです。この問題を解決するために、以下のいずれかの方法を使用できます：
* Word2Vecモデルを私たちの語彙で再学習する。
* 事前学習済みのWord2Vecモデルの語彙を使用してデータセットをロードする。データセットをロードする際に使用する語彙は指定可能です。

後者の方法の方が簡単そうなので、これを実装してみましょう。まず、Word2Vec埋め込みから取得した指定語彙を使用して`TextVectorization`層を作成します：


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

gensimの単語埋め込みライブラリには便利な関数`get_keras_embeddings`が含まれており、対応するKeras埋め込み層を自動的に作成してくれます。


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


私たちがより高い精度を見ていない理由の一つは、データセットのいくつかの単語が事前学習済みのGloVe語彙に欠けているため、それらが事実上無視されているからです。これを克服するために、データセットに基づいて独自の埋め込みを訓練することができます。


## 文脈的埋め込み

Word2Vecのような従来の事前学習済み埋め込み表現の主な制限の1つは、単語の意味をある程度捉えることはできても、異なる意味を区別することができない点です。この制限は、下流のモデルで問題を引き起こす可能性があります。

例えば、単語「play」は以下の2つの文で異なる意味を持っています：
- 私は劇場で**play**（劇）を観ました。
- ジョンは友達と**play**（遊ぶ）をしたいと思っています。

先ほど説明した事前学習済み埋め込みは、単語「play」の両方の意味を同じ埋め込みで表現します。この制限を克服するためには、**言語モデル**に基づいて埋め込みを構築する必要があります。この言語モデルは、大規模なテキストコーパスで訓練されており、単語が異なる文脈でどのように組み合わされるかを*理解*しています。文脈的埋め込みについて詳しく説明することはこのチュートリアルの範囲外ですが、次のユニットで言語モデルについて話す際に再び取り上げます。



---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当社は責任を負いません。
